#### Infill NOTEBOOK

This notebook validates the Bayesian multi-objective adaptive infill strategy.

**Notes**: the tests are performed for multi-fidelity multi-objective optimization similarly to work from Charayron et al. [(1)](https://www.sciencedirect.com/science/article/pii/S1270963823005692?via%3Dihub).

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
from typing import Any

from aero_optim.mf_sm.mf_models import get_model, get_sampler, MultiObjectiveModel, MfDNN, MfSMT
from aero_optim.mf_sm.mf_infill import compute_pareto, minimize_LCB, maximize_ED, maximize_MPI_BO, maximize_PI_BO
from pymoo.problems import get_problem

ZDT1 multi-fidelity multi-objective functions

In [ ]:
def f1_hf(x):
    f1 = x[:, 0]
    return f1

def f2_hf(x):
    n_var = x.shape[-1]
    u = 1 + 9.0 / (n_var - 1) * np.sum(x[:, 1:], axis=1)
    v = 1 - np.sqrt(f1_hf(x) / u)
    f2 = u * v
    return f2

def f_hf(x):
    return np.column_stack([f1_hf(x), f2_hf(x)])

def f1_lf(x):
    f1 = 0.9 * x[:, 0] + 0.1
    return f1

def f2_lf(x):
    n_var = x.shape[-1]
    u = 1 + 9.0 / (n_var - 1) * np.sum(x[:, 1:], axis=1)
    v = 1 - np.sqrt(f1_hf(x) / u)
    return (0.8 * u - 0.2) * (1.2 * v + 0.2)

def f_lf(x):
    return np.column_stack([f1_lf(x), f2_lf(x)])

Bayesian infill sample computation function

In [ ]:
def compute_bayesian_infill(
        model: MfDNN | MultiObjectiveModel,
        infill_lf_size: int,
        infill_nb_gen: int,
        n_design: int,
        bound: tuple[Any],
        seed: int
) -> np.ndarray:
    """
    **Computes** the low fidelity Bayesian infill candidates.
    """
    assert isinstance(model, MultiObjectiveModel)
    # Probability of Improvement
    infill_lf = maximize_MPI_BO(model, n_design, bound, seed, infill_nb_gen)
    # Lower Confidence Bound /objective 1
    assert isinstance(model.models[0], MfSMT)
    infill_lf_LCB_1 = minimize_LCB(model.models[0], n_design, bound, seed, infill_nb_gen)
    infill_lf = np.vstack((infill_lf, infill_lf_LCB_1))
    # Lower Confidence Bound /objective 2
    assert isinstance(model.models[1], MfSMT)
    infill_lf_LCB_2 = minimize_LCB(model.models[1], n_design, bound, seed, infill_nb_gen)
    infill_lf = np.vstack((infill_lf, infill_lf_LCB_2))
    # max-min Euclidean Distance
    current_DOE = model.get_DOE()
    current_DOE = np.vstack((current_DOE, infill_lf))
    for _ in range(infill_lf_size - 3):
        infill_lf_ED = maximize_ED(current_DOE, n_design, bound, seed, infill_nb_gen)
        infill_lf = np.vstack((infill_lf, infill_lf_ED))
        current_DOE = np.vstack((current_DOE, infill_lf_ED))
    return infill_lf

#### 1. Custom Bayesian infill strategy

The Bayesian infill input variables are:

- `seed` the random seed
- `dim` the dimension of the problem
- `n_lf` the number of initial low-fidelity samples to draw
- `n_hf` the number of initial high-fidelity samples to draw
- `n_iter` the number of infill steps
- `infill_lf_size` the number of low-fidelity samples to compute at each infill step
- `infill_nb_gen` the number of generations of the sub-optimization executions

**Note**: the low- / high-fidelity infill ratio is 10 to 1

In [ ]:
seed = 123
dim = 6
n_lf = 12
n_hf = 6
n_iter = 15
infill_lf_size = 10
infill_nb_gen = 50

Builds the nested LHS sampler

In [ ]:
mf_sampler = get_sampler(dim, bounds=[0, 1], seed=seed, nested_doe=True)
x_lf, x_hf = mf_sampler.sample_mf(n_lf, n_hf)
y_lf = f_lf(x_lf)
y_hf = f_hf(x_hf)

Builds the multi-objective co-kriging model

In [ ]:
model1 = get_model(model_name="mfsmt", dim=dim, config_dict={}, outdir="", seed=seed)
model2 = get_model(model_name="mfsmt", dim=dim, config_dict={}, outdir="", seed=seed)

In [ ]:
mo_model = MultiObjectiveModel([model1, model2])
mo_model.set_DOE(x_lf=x_lf, x_hf=x_hf, y_lf=[y_lf[:, 0], y_lf[:, 1]], y_hf=[y_hf[:, 0], y_hf[:, 1]])
mo_model.train()

Bayesian adaptive infill loop

**Note**: this should take around 4 minutes

In [ ]:
bound = [0, 1]
pareto_list = [compute_pareto(mo_model.models[0].y_hf_DOE, mo_model.models[1].y_hf_DOE)]
for _ in range(n_iter):
    x_lf_infill = compute_bayesian_infill(mo_model, infill_lf_size, infill_nb_gen, dim, bound, seed)
    y_lf_infill = f_lf(x_lf_infill)
    x_hf_infill = x_lf_infill[0]
    y_hf_infill = f_hf(x_hf_infill.reshape(1, -1))
    print(f"iter {_}, new x_f {x_hf_infill}, new y_hf {y_hf_infill}")
    mo_model.set_DOE(x_lf=x_lf_infill, y_lf=[y_lf_infill[:, 0], y_lf_infill[: ,1]], x_hf=x_hf_infill, y_hf=[y_hf_infill[:, 0], y_hf_infill[:, 1]])
    mo_model.train()
    print("model retrained")
    pareto_list.append(compute_pareto(mo_model.models[0].y_hf_DOE, mo_model.models[1].y_hf_DOE))

Compute the analytical Pareto front

In [ ]:
problem = get_problem("zdt1")
true_pareto = problem.pareto_front()

Bayesian adaptive infill results are plotted

In [ ]:
fig, ax = plt.subplots()
ax.plot(true_pareto[:, 0], true_pareto[:, 1], color="r", label="true pareto")
cm = plt.get_cmap('viridis')
COLORS = [cm(ii * 10) for ii in range(len(pareto_list) + 1)]
ax.scatter(pareto_list[-1][:, 0], pareto_list[-1][:, 1], color=COLORS[-1], marker="d", label="final Pareto")
ax.scatter(mo_model.models[0].y_hf_DOE[:n_hf], mo_model.models[1].y_hf_DOE[:n_hf], color="k", facecolors="None", label="initial DOE")
ax.scatter(mo_model.models[0].y_hf_DOE[n_hf:], mo_model.models[1].y_hf_DOE[n_hf:], color="r", facecolors="None", label="hf infills")
ax.set(xlabel='$y_1$', ylabel='$y_2$')
plt.legend()

#### 2. Maximal Probability of Improvement infill strategy

The MPI infill input variables are:

- `seed` the random seed
- `dim` the dimension of the problem
- `n_lf` the number of initial low-fidelity samples to draw
- `n_hf` the number of initial high-fidelity samples to draw
- `n_iter` the number of infill steps
- `infill_nb_gen` the number of generations of the sub-optimization executions

**Note**: at each step a unique low- and high-fidelity sample is computed

In [ ]:
seed = 123
dim = 6
n_lf = 12
n_hf = 6
n_iter = 15
infill_nb_gen = 50

Builds the nested LHS sampler

In [ ]:
mf_sampler = get_sampler(dim, bounds=[0, 1], seed=seed, nested_doe=True)
x_lf, x_hf = mf_sampler.sample_mf(n_lf, n_hf)
y_lf = f_lf(x_lf)
y_hf = f_hf(x_hf)

Builds the multi-objective co-kriging model

In [ ]:
model1 = get_model(model_name="mfsmt", dim=dim, config_dict={}, outdir="", seed=seed)
model2 = get_model(model_name="mfsmt", dim=dim, config_dict={}, outdir="", seed=seed)

In [ ]:
mo_model = MultiObjectiveModel([model1, model2])
mo_model.set_DOE(x_lf=x_lf, x_hf=x_hf, y_lf=[y_lf[:, 0], y_lf[:, 1]], y_hf=[y_hf[:, 0], y_hf[:, 1]])
mo_model.train()

MPI adaptive infill loop

**Note**: this should take around 1 minute

In [ ]:
bound = [0, 1]
pareto_list = [compute_pareto(mo_model.models[0].y_hf_DOE, mo_model.models[1].y_hf_DOE)]
for _ in range(n_iter):
    x_lf_infill = maximize_MPI_BO(mo_model, dim, bound, seed, infill_nb_gen)
    y_lf_infill = f_lf(x_lf_infill.reshape(1, -1))
    x_hf_infill = x_lf_infill
    y_hf_infill = f_hf(x_hf_infill.reshape(1, -1))
    print(f"iter {_}, new x_f {x_hf_infill}, new y_hf {y_hf_infill}")
    mo_model.set_DOE(x_lf=x_lf_infill, y_lf=[y_lf_infill[:, 0], y_lf_infill[: ,1]], x_hf=x_hf_infill, y_hf=[y_hf_infill[:, 0], y_hf_infill[:, 1]])
    mo_model.train()
    print("model retrained")
    pareto_list.append(compute_pareto(mo_model.models[0].y_hf_DOE, mo_model.models[1].y_hf_DOE))

MPI adaptive infill results are plotted

In [ ]:
fig, ax = plt.subplots()
ax.plot(true_pareto[:, 0], true_pareto[:, 1], color="r", label="true pareto")
cm = plt.get_cmap('viridis')
COLORS = [cm(ii * 10) for ii in range(len(pareto_list) + 1)]
ax.scatter(pareto_list[-1][:, 0], pareto_list[-1][:, 1], color=COLORS[-1], marker="d", label="final pareto")
ax.scatter(mo_model.models[0].y_hf_DOE[:n_hf], mo_model.models[1].y_hf_DOE[:n_hf], color="k", facecolors="None", label="initial DOE")
ax.scatter(mo_model.models[0].y_hf_DOE[n_hf:], mo_model.models[1].y_hf_DOE[n_hf:], color="r", facecolors="None", label="infill")
ax.set(xlabel='$y_1$', ylabel='$y_2$')
plt.legend()